

## Objectives:
Using this notebook we will:

1.  Understand the Spacex DataSet
2.  Load the dataset into the corresponding table in a database
3.  Execute SQL queries for explanatory data analysis


We start by installing and importing the required packages.

In [1]:
#!pip install sqlalchemy==1.3.9
#!pip install ipython-sql
#!pip install -q pandas==1.1.5

import sqlite3
import pandas as pd

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
# This dataset is provided by the IBM Data Science course
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

Note:This below code is added to remove blank rows from table


In [6]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## **Exploratory Data Analysis**

**Now we can write and execute some SQL queries to understand our data and gain some insights**







##### Let's display the names of the unique launch sites  in the space mission


In [7]:
%%sql
Select Distinct "Launch_Site"
From SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



#####  Let's display 5 records where launch sites begin with the string 'CCA' 


In [8]:
%%sql
Select * From SPACEXTABLE
Where "Launch_Site" Like "CCA%"
Limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt







##### Let's display the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%%sql
Select Sum(PAYLOAD_MASS__KG_) From SPACEXTABLE
Where Customer == "NASA (CRS)";

 * sqlite:///my_data1.db
Done.


Sum(PAYLOAD_MASS__KG_)
45596







##### Let's display average payload mass carried by booster version F9 v1.1


In [10]:
%%sql
Select Avg(PAYLOAD_MASS__KG_) From SPACEXTABLE
Where Booster_Version == "F9 v1.1"

 * sqlite:///my_data1.db
Done.


Avg(PAYLOAD_MASS__KG_)
2928.4




##### Let's show the date when the first succesful landing outcome in ground pad was acheived.




In [11]:
%%sql
Select Min(Date) From SPACEXTABLE
Where Landing_Outcome == "Success (ground pad)";

 * sqlite:///my_data1.db
Done.


Min(Date)
2015-12-22




##### Let's list the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [12]:
%%sql
Select Booster_Version From SPACEXTABLE
Where (Landing_Outcome == "Success (drone ship)")
and (PAYLOAD_MASS__KG_ > 4000)
and (PAYLOAD_MASS__KG_ < 6000);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2







##### Let's list the total number of successful and failure mission outcomes


In [13]:
%%sql
Select Count(Mission_Outcome) as OutcomeCount, 'Success' as OutcomeType
From SPACEXTABLE
Where Mission_Outcome Like "%Success%"

Union All

Select Count(Mission_Outcome) as OutcomeCount, 'Failure' as OutcomeType
From SPACEXTABLE
Where Mission_Outcome Like "%Failure%";

 * sqlite:///my_data1.db
Done.


OutcomeCount,OutcomeType
100,Success
1,Failure






##### Let's list the   names of the booster_versions which have carried the maximum payload mass. We can use a subquery


In [14]:
%%sql
Select Booster_Version From SPACEXTABLE
Where PAYLOAD_MASS__KG_ == (Select Max(PAYLOAD_MASS__KG_) From SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6



##### Let's list the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.



In [15]:
%%sql
Select substr(Date,6,2) as month, Landing_Outcome, Booster_Version, Launch_Site
From SPACEXTABLE
Where (Landing_Outcome == "Failure (drone ship)")
and (substr(Date,0,5) == "2015");


 * sqlite:///my_data1.db
Done.


month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40




##### Let's rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [16]:
%%sql
Select Landing_Outcome, Count(*) as Count From SPACEXTABLE
Where (Date < '2017-03-21')  and (Date > '2010-06-03')
Group By Landing_Outcome
Order By Count Desc;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


The above queries helped us gain some insights from our data. However, keep in mind that a similar analysis can be performed using Python language instead of SQL.